In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
import torch.optim as optim 

In [9]:

# Hype-parameters
num_epochs = 4 # how many times we are running 
batch_size = 32 # 
learning_rate = .001 # 

mnist = datasets.MNIST(
    root='./data', # where to store data
    train=True, # tell the code it is training data
    download=True, # download the data
    transform=transforms.ToTensor() # transform dataset to tensor directly (no preprocessing)
) # import data
print(len(mnist))

mnist_train, mnist_test = torch.utils.data.random_split(mnist, [0.8, .2]) # split data 80/20
print(len(mnist_train))
print(len(mnist_test))  


60000
48000
12000


In [10]:
batch_size = 32

# init dataloaders to load batches into model
train_dl  = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True) 

test_dl   = torch.utils.data.DataLoader(mnist_test, batch_size=10000, shuffle=False ) 


In [11]:
# function to measure accuracy, confusion matrix, precision, recall ,f1 (metrics to measure classification)
def print_metrics_function(y_test, y_pred):
    print('Accuracy: %.6f' % accuracy_score(y_test, y_pred))
    confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
    print("Confusion Matrix:")
    print(confmat)
    print('Precision: %.3f' % precision_score(y_true=y_test, y_pred=y_pred, average='weighted'))
    print('Recall: %.3f' % recall_score(y_true=y_test, y_pred=y_pred, average='weighted'))
    f1_measure = f1_score(y_true=y_test, y_pred=y_pred, average='weighted')
    print('F1-mesure: %.3f' % f1_measure)
    return f1_measure

In [12]:
class Classifier6(nn.Module):

    def __init__(self):
        super().__init__()

        self.model = nn.Sequential(
            ## Convolitional Layer 1
                nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=1), # 1 input 16 filters, padding so same dim
                nn.ReLU(), # ReLU introduce non-linearity
                nn.MaxPool2d(2, 2), #pool
 
                ## Convolutional Layer 2
                nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=1), # 16 inputs 32 filters
                nn.ReLU(),
                nn.MaxPool2d(2, 2),   
 
                ## feed forward layer w/ 1024 neurons, regular layer
                nn.Flatten(),
                nn.Linear(800, 1024),    ## see how to get 800 below on last cell
                nn.ReLU(),

                nn.Linear(1024, 10), # maps to output w/ 10 classes
                nn.LogSoftmax(dim=1)
        )
   
    def forward(self, inputs):
        return self.model(inputs)
        
def training_loop( num_epochs, model, loss_fn, opt):

    losses_list = []
    
    for epoch in range(num_epochs):
        for xb, yb in train_dl:
            
            ## print( xb.shape )   ## check this comes out as [N, 1, 28, 28]
            ## yb = torch.squeeze(yb, dim=1)
            
            y_pred = model(xb)
            loss   = loss_fn(y_pred, yb)
    
            opt.zero_grad()
            loss.backward()
            opt.step()
            
        if epoch % 1 == 0:
            print(epoch, "loss=", loss)
            losses_list.append(  loss  )
            
    return losses_list

In [13]:
model = Classifier6() # create our model

opt = torch.optim.Adam(model.parameters(), lr = learning_rate) # optimizer that does steps

loss_fn = nn.CrossEntropyLoss() # type of loss func

my_losses_list = training_loop(num_epochs, model, loss_fn, opt)

0 loss= tensor(0.0075, grad_fn=<NllLossBackward0>)
1 loss= tensor(0.0017, grad_fn=<NllLossBackward0>)
2 loss= tensor(0.1347, grad_fn=<NllLossBackward0>)
3 loss= tensor(0.0706, grad_fn=<NllLossBackward0>)


In [26]:
with torch.no_grad(): # detach grad tracking for tensors
    for x_real, y_real in test_dl:
        y_pred = model(  x_real  )
        
        vals, indeces = torch.max( y_pred, dim=1  )
        preds = indeces
        print_metrics_function(y_real, preds)

Accuracy: 0.986100
Confusion Matrix:
[[ 982    0    0    0    0    1    5    0    0    0]
 [   0 1122    4    0    0    0    1    0    0    0]
 [   1    1 1013    1    1    0    0    3    1    0]
 [   2    2    4  980    0    3    0    5    0    1]
 [   1    1    0    0  964    0    1    0    1    1]
 [   0    0    0    1    1  894    4    0    0    0]
 [   0    0    0    0    1    6  999    0    1    0]
 [   0    8    2    0    0    1    0  976    0    9]
 [   5    8    6    1    2    9    5    0  950    2]
 [   3    1    0    1   13    2    0    4    2  981]]
Precision: 0.986
Recall: 0.986
F1-mesure: 0.986
Accuracy: 0.989000
Confusion Matrix:
[[197   0   0   0   0   0   1   0   0   0]
 [  1 233   0   0   0   0   0   0   0   0]
 [  0   0 204   0   0   0   0   0   0   0]
 [  0   0   0 195   0   2   0   0   1   0]
 [  0   0   0   0 208   0   1   0   0   1]
 [  0   0   0   0   0 198   0   0   0   0]
 [  0   0   0   1   1   0 172   0   0   0]
 [  0   0   1   0   0   0   0 200   0   0]
 [ 

In [ ]:
from sklearn.model_selection import KFold
import torch
import torch.optim as optim
import torch.nn as nn

def k_fold_valid(model_class, train_dataloader, num_folds=5, num_epochs=10, batch_size=32):
    kfold = KFold(n_splits=num_folds, shuffle=True)
    
    # Placeholder to store results from each fold
    fold_results = []

    for fold, (train_idx, val_idx) in enumerate(kfold.split(train_dataloader.dataset)):
        print(f"Fold {fold + 1}/{num_folds}")

        # Split the DataLoader dataset into training and validation sets for this fold
        # Create subsets for training and validation by selecting indices from the DataLoader
        train_subset = torch.utils.data.Subset(train_dataloader.dataset, train_idx)
        val_subset = torch.utils.data.Subset(train_dataloader.dataset, val_idx)

        # Create new DataLoaders for each fold
        train_loader = torch.utils.data.DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = torch.utils.data.DataLoader(val_subset, batch_size=batch_size, shuffle=False)
        
        # Initialize the model and optimizer
        model = model_class()  # Instantiate the model class
        criterion = nn.CrossEntropyLoss()  # Loss function
        optimizer = optim.Adam(model.parameters(), lr=0.001)  # Optimizer

        # Training loop
        for epoch in range(num_epochs):
            model.train()  # Set the model to training mode
            running_loss = 0.0
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

            avg_train_loss = running_loss / len(train_loader)
            print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}")
        
        # Validation loop
        model.eval()  # Set the model to evaluation mode
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = correct / total
        print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

        fold_results.append(val_accuracy)

    # Calculate and display the average validation accuracy
    avg_accuracy = sum(fold_results) / num_folds
    print(f"Average Validation Accuracy over {num_folds} folds: {avg_accuracy * 100:.2f}%")

k_fold_valid(
    model_class = Classifier6, 
    train_dataloader = mnist_train,
    num_folds = 5,
    num_epochs = 10,
    batch_size = batch_size
    )


Fold 1/5
Epoch 1/10, Train Loss: 0.1384
Epoch 2/10, Train Loss: 0.0435
Epoch 3/10, Train Loss: 0.0298
Epoch 4/10, Train Loss: 0.0220
Epoch 5/10, Train Loss: 0.0162
Epoch 6/10, Train Loss: 0.0132
Epoch 7/10, Train Loss: 0.0112
Epoch 8/10, Train Loss: 0.0096
Epoch 9/10, Train Loss: 0.0086
Epoch 10/10, Train Loss: 0.0070
Validation Accuracy: 98.91%
Fold 2/5
Epoch 1/10, Train Loss: 0.1336
Epoch 2/10, Train Loss: 0.0427
Epoch 3/10, Train Loss: 0.0308
Epoch 4/10, Train Loss: 0.0206
Epoch 5/10, Train Loss: 0.0173
Epoch 6/10, Train Loss: 0.0138
Epoch 7/10, Train Loss: 0.0112
Epoch 8/10, Train Loss: 0.0078
Epoch 9/10, Train Loss: 0.0099
Epoch 10/10, Train Loss: 0.0087
Validation Accuracy: 98.85%
Fold 3/5
Epoch 1/10, Train Loss: 0.1387
Epoch 2/10, Train Loss: 0.0465
Epoch 3/10, Train Loss: 0.0318
Epoch 4/10, Train Loss: 0.0214
Epoch 5/10, Train Loss: 0.0175
Epoch 6/10, Train Loss: 0.0148
Epoch 7/10, Train Loss: 0.0120
Epoch 8/10, Train Loss: 0.0110
Epoch 9/10, Train Loss: 0.0074
Epoch 10/10, Tra

k-cross valid results
Fold 1/5
Epoch 1/10, Train Loss: 0.1384
Epoch 2/10, Train Loss: 0.0435
Epoch 3/10, Train Loss: 0.0298
Epoch 4/10, Train Loss: 0.0220
Epoch 5/10, Train Loss: 0.0162
Epoch 6/10, Train Loss: 0.0132
Epoch 7/10, Train Loss: 0.0112
Epoch 8/10, Train Loss: 0.0096
Epoch 9/10, Train Loss: 0.0086
Epoch 10/10, Train Loss: 0.0070
Validation Accuracy: 98.91%
Fold 2/5
Epoch 1/10, Train Loss: 0.1336
Epoch 2/10, Train Loss: 0.0427
Epoch 3/10, Train Loss: 0.0308
Epoch 4/10, Train Loss: 0.0206
Epoch 5/10, Train Loss: 0.0173
Epoch 6/10, Train Loss: 0.0138
Epoch 7/10, Train Loss: 0.0112
Epoch 8/10, Train Loss: 0.0078
Epoch 9/10, Train Loss: 0.0099
Epoch 10/10, Train Loss: 0.0087
Validation Accuracy: 98.85%
Fold 3/5
Epoch 1/10, Train Loss: 0.1387
Epoch 2/10, Train Loss: 0.0465
Epoch 3/10, Train Loss: 0.0318
Epoch 4/10, Train Loss: 0.0214
Epoch 5/10, Train Loss: 0.0175
Epoch 6/10, Train Loss: 0.0148
Epoch 7/10, Train Loss: 0.0120
Epoch 8/10, Train Loss: 0.0110
Epoch 9/10, Train Loss: 0.0074
Epoch 10/10, Train Loss: 0.0100
Validation Accuracy: 99.22%
Fold 4/5
Epoch 1/10, Train Loss: 0.1346
Epoch 2/10, Train Loss: 0.0437
Epoch 3/10, Train Loss: 0.0314
Epoch 4/10, Train Loss: 0.0212
Epoch 5/10, Train Loss: 0.0175
Epoch 6/10, Train Loss: 0.0143
Epoch 7/10, Train Loss: 0.0122
Epoch 8/10, Train Loss: 0.0093
Epoch 9/10, Train Loss: 0.0082
Epoch 10/10, Train Loss: 0.0082
Validation Accuracy: 98.90%
Fold 5/5
Epoch 1/10, Train Loss: 0.1428
Epoch 2/10, Train Loss: 0.0490
Epoch 3/10, Train Loss: 0.0321
Epoch 4/10, Train Loss: 0.0238
Epoch 5/10, Train Loss: 0.0196
Epoch 6/10, Train Loss: 0.0146
Epoch 7/10, Train Loss: 0.0129
Epoch 8/10, Train Loss: 0.0117
Epoch 9/10, Train Loss: 0.0095
Epoch 10/10, Train Loss: 0.0080
Validation Accuracy: 98.87%
Average Validation Accuracy over 5 folds: 98.95%

In [ ]:
# save model
torch.save(model.state_dict(), "team6_final_weights.pth")   

In [37]:
# test handwritten digits
from PIL import Image
import numpy as np


# Load the image, replace with custom file path
image_path = [
    "C:/Eng96ML/img1.jpg",
    "C:/Eng96ML/img2.jpg",
    "C:/Eng96ML/img3.jpg"
    #"C:/Eng96ML/img4.jpg",
    #"C:/Eng96ML/img5.jpg",
    #"C:/Eng96ML/img6.jpg",
    #"C:/Eng96ML/img7.jpg",
    #"C:/Eng96ML/img8.jpg",
    #"C:/Eng96ML/img9.jpg",
    #"C:/Eng96ML/img10.jpg"
]  
for path in image_path:
    image = Image.open(path)

    # convert to greyscale & resize
    image = image.convert("L")
    image = image.resize((28,28))

    # convert to tensor
    transform = transforms.ToTensor()
    image_tr = transform(image)
    image_tr = image_tr.unsqueeze(dim=0) # add batch dimension

    pred = model(image_tr)
    vals, indeces = torch.max( pred, dim=1  )
    preds = indeces
    print(preds)

tensor([6])
tensor([4])
tensor([2])


In [ ]:
# test handwritten digit

# code so image is in correct format

y_pred = model() # image in correct format inputted

In [20]:
# commented out, took out validation idk if we need

#from torch.utils.data import DataLoader
#validation_loader = DataLoader(mnist_valid, batch_size=64, shuffle=False)

# Iterate through the DataLoader to validate
#for X_valid, y_valid in validation_loader:
    # Use X_valid and y_valid for validation here
    #break  # Remove this if processing the entire validation set
#y_valid_pred = model(X_valid)  # Predictions on validation set

#val_accuracy = accuracy_score(y_valid, y_valid_pred)
#print(f"Validation Accuracy: {val_accuracy:.2f}")